In [1]:
# import libraries 
import pandas as pd
import numpy as np
import math
from numpy.linalg import norm

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from pickle import dump, load

pd.set_option('display.max_columns', None)

np.random_state = 42

## Load in `sd_pp` Dataset

In [2]:
# load in df with the cluster labels
sd_pp = pd.read_csv('data/sd_pp', index_col = 0)
sd_pp

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,-1.152181,-0.561263,-0.534033,-0.728248,2.546108,0.605540,1.504754,-0.909516,-0.562984,-0.490416,3.516369,-0.408425,0.513223,-0.764034,-0.238620,0.038201,-0.972836,-0.581030,-0.581030,0.372804,0.368466,0.464525,0.272019,0.278668,0.342562,0.598182,0.0,0.970300,0.0,-0.16778,-0.183109
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Get the `listing_url`

In [3]:
# load in df with the listing urls
sd_listings_url = pd.read_csv('data/sd_modeling_with_urls', index_col= 0)
sd_listings_url = sd_listings_url[['listing_url']]
sd_listings_url = sd_listings_url.reset_index(drop = 'index')
sd_listings_url

,listing_url
0,https://www.airbnb.com/rooms/33159143
1,https://www.airbnb.com/rooms/17138468
2,https://www.airbnb.com/rooms/21898446
3,https://www.airbnb.com/rooms/25948680
4,https://www.airbnb.com/rooms/1756516
...,...
13034,https://www.airbnb.com/rooms/20473217
13035,https://www.airbnb.com/rooms/17975465
13036,https://www.airbnb.com/rooms/4227110
13037,https://www.airbnb.com/rooms/17975942


## Create a new dataframe for comparison listing

**Select a listing**

In [4]:
# selects a random listing from sd_clustered
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
12864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.832322,4.410786,5.324477,7.163287,5.117673,1.548534,-0.555421,2.194861,4.172152,1.320389,-0.174899,0.389161,0.513223,-0.764034,-0.23862,0.038201,-0.972836,0.485043,0.485043,-0.247675,-1.157717,-0.88476,0.272019,-1.562972,0.342562,-0.717556,0.0,0.9703,0.0,-0.16778,-0.183109


### Convert to Arrays

In [5]:
# convert single listing to an array
random_listing_array = random_listing.values

# convert all listings to an array
sd_pp_array = sd_pp.values

In [6]:
print(random_listing_array.shape)
print(sd_pp_array.shape) 

(1, 240)
(13039, 240)


## Determine Cosine Similarity

In [7]:
# define two lists or arrays to compare
A = np.squeeze(np.asarray(sd_pp_array))
B = np.squeeze(np.asarray(random_listing_array))
print("A:\n", A)
print("B:\n", B)
 
# compute cosine similarity
cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))
print("Cosine Similarity:\n", cosine)

A:
 [[ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 ...
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]]
B:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.     

### Add Cosine Similarity to `sd_clustered`

In [8]:
# load in df with the cluster labels
sd_clustered = pd.read_csv('data/sd_clustered', index_col = 0)
sd_clustered.reset_index(drop = 'index')

,id,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,33159143,2,32.91736,-117.07635,Scripps Ranch,92131,House,Private room,1,1.0,1.0,1,Real Bed,1100.0,1100.0,1000.0,0.0,1.0,0.0,60.0,90.0,within a few hours,100.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
1,17138468,3,32.84067,-117.27443,La Jolla,92037,Apartment,Entire home/apt,1,2.0,2.0,3,Real Bed,1400.0,1400.0,0.0,0.0,1.0,0.0,30.0,1125.0,no response,0.0,0.0,1.0,1.0,0.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
2,21898446,3,32.79797,-117.24250,Pacific Beach,92109,Townhouse,Private room,1,1.0,1.0,1,Real Bed,1250.0,1250.0,0.0,0.0,1.0,0.0,30.0,30.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
3,25948680,3,32.77545,-117.05923,College Area,92120,Apartment,Entire home/apt,1,1.0,1.0,1,Real Bed,1150.0,1150.0,0.0,0.0,1.0,0.0,31.0,40.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
4,1756516,2,32.84619,-117.27558,La Jolla,92037,Condominium,Private room,1,1.0,1.0,1,Real Bed,110.0,110.0,200.0,0.0,1.0,50.0,180.0,365.0,within a day,67.0,0.0,6.0,1.0,1.0,2.0,4.0,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0.0,0.0,0.0,strict_14_with_grace_period,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13034,20473217,0,32.78399,-117.25391,Mission Bay,92109,Villa,Entire home/apt,20,7.0,8.0,11,Real Bed,3000.0,3000.0,3000.0,0.0,20.0,0.0,1.0,1125.0,within an hour,95.0,1.0,86.0,1.0,1.0,0.0,0.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,luxury_moderate,0.0,0.0
13035,17975465,0,32.80892,-117.26588,La Jolla,92037,House,Entire home/apt,20,7.0,10.0,13,Real Bed,2823.0,2823.0,1000.0,874.0,20.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,6.0,12.0,83.0,8.0,8.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
13036,4227110,0,32.79395,-117.23492,Mission Bay,92109,House,Entire home/apt,20,2.0,4.0,10,Real Bed,208.0,208.0,1000.0,332.0,4.0,25.0,1.0,1125.0,within an hour,96.0,0.0,11.0,1.0,1.0,142.0,284.0,92.0,9.0,9.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
13037,17975942,0,32.83598,-117.27878,La Jolla,92037,House,Entire home/apt,21,5.5,7.0,13,Real Bed,1305.0,1305.0,1000.0,800.0,21.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,4.0,8.0,80.0,9.0,8.0,10.0,8.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0


#### Adding in `listing_url`

In [9]:
# merge on index
sd_clustered = sd_clustered.join(sd_listings_url)

In [10]:
# add similarity 
rec = sd_clustered.copy()
rec['similarity'] = pd.DataFrame(cosine).values

In [11]:
# reorder column names
rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]

## Get Recommendations!

In [12]:
# sort by highest similarity
rec.sort_values(by = ['similarity'], ascending = False).head(6)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
12876,23257256,https://www.airbnb.com/rooms/32998014,1.000000,0,32.88163,-117.24742,Torrey Pines,92037,House,Entire home/apt,16,5.5,8.0,16,Real Bed,1985.0,1985.0,0.0,325.0,12.0,50.0,2.0,1125.0,within an hour,100.0,0.0,1.0,1.0,0.0,64.0,128.0,94.0,9.0,9.0,10.0,9.0,10.0,9.0,0.0,1.0,0.0,moderate,0.0,0.0
12915,33716671,https://www.airbnb.com/rooms/32603515,0.910902,0,32.77342,-117.25202,Mission Bay,92109,Condominium,Entire home/apt,16,4.0,5.0,15,Real Bed,1999.0,1999.0,300.0,280.0,12.0,20.0,1.0,1125.0,within an hour,98.0,0.0,24.0,1.0,0.0,0.0,0.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
13047,17975465,NaN,0.887594,0,32.80892,-117.26588,La Jolla,92037,House,Entire home/apt,20,7.0,10.0,13,Real Bed,2823.0,2823.0,1000.0,874.0,20.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,6.0,12.0,83.0,8.0,8.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
12869,21736886,https://www.airbnb.com/rooms/637308,0.886867,0,32.76330,-117.25102,Mission Bay,92109,House,Entire home/apt,16,6.0,6.0,21,Real Bed,1800.0,1800.0,0.0,800.0,16.0,0.0,3.0,1125.0,within a day,60.0,0.0,2.0,1.0,0.0,1.0,2.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
12944,2201627,https://www.airbnb.com/rooms/32164162,0.877581,0,32.72552,-117.16209,Park West,92101,House,Entire home/apt,16,5.0,10.0,17,Real Bed,1200.0,1200.0,1000.0,500.0,16.0,25.0,1.0,1125.0,within an hour,100.0,1.0,9.0,1.0,1.0,50.0,100.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,strict_14_with_grace_period,0.0,0.0
12960,32522521,https://www.airbnb.com/rooms/28774764,0.871820,0,32.67219,-116.94402,Rolling Hills Ranch,91914,House,Entire home/apt,16,5.5,7.0,16,Real Bed,988.0,988.0,750.0,265.0,10.0,25.0,1.0,1125.0,within a few hours,100.0,1.0,5.0,1.0,0.0,8.0,16.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,strict_14_with_grace_period,0.0,0.0


### Get Recommendations Function

In [13]:
# selects a random listing from sd_clustered
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
2795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.819881,-0.561263,-0.534033,-0.728248,-0.449693,-0.419501,-0.555421,-0.728029,-0.562984,-0.490416,-0.174899,-0.472386,0.513223,-0.764034,-0.23862,0.038201,1.027922,-0.181252,-0.181252,0.527923,0.368466,0.464525,0.272019,0.278668,0.342562,0.598182,0.0,-1.030609,0.0,-0.16778,-0.183109


In [14]:
def get_recommendations(df, listing):
    """
    Takes in preprocessed dataframe and selected listing as inputs and gives top 5 (including listing)
    recommendations based on cosine similarity. 
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # convert single listing to an array
    listing_array = listing.values

    # convert all listings to an array
    df_array = df.values
    
    # get arrays into a single dimension
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # compute cosine similarity 
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # add similarity into recommendations df and reset the index
    rec = sd_clustered.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # reorder column names
    rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(6)


In [15]:
get_recommendations(sd_pp, random_listing)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
2795,18991166,https://www.airbnb.com/rooms/22655070,1.000000,2,32.73838,-117.16360,Midtown,92103,Apartment,Entire home/apt,2,1.0,1.0,1,Real Bed,69.0,138.0,0.0,19.0,1.0,0.0,2.0,7.0,within an hour,100.0,0.0,1.0,1.0,1.0,24.0,48.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
276,16930468,https://www.airbnb.com/rooms/4714055,0.957259,2,32.74624,-117.17718,Midtown,92103,Apartment,Entire home/apt,1,1.0,1.0,1,Real Bed,76.0,152.0,0.0,0.0,1.0,0.0,2.0,2.0,within an hour,100.0,0.0,1.0,1.0,1.0,28.0,56.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0
5287,34288376,https://www.airbnb.com/rooms/14531646,0.924120,2,32.74673,-117.16350,Midtown,92103,Apartment,Entire home/apt,3,1.0,1.0,1,Real Bed,80.0,160.0,0.0,25.0,1.0,0.0,2.0,1125.0,within a few hours,100.0,0.0,2.0,1.0,1.0,4.0,8.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
6091,35648614,https://www.airbnb.com/rooms/26926812,0.840659,2,32.74768,-117.14762,Midtown,92103,Apartment,Entire home/apt,4,1.0,1.0,2,Real Bed,98.0,1176.0,250.0,74.0,1.0,25.0,12.0,29.0,within an hour,100.0,0.0,2.0,1.0,1.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
8101,28608521,https://www.airbnb.com/rooms/21622025,0.809570,2,32.74164,-117.15113,Midtown,92103,Guesthouse,Entire home/apt,4,1.5,1.0,1,Real Bed,92.0,92.0,300.0,100.0,1.0,0.0,1.0,11.0,within a few hours,100.0,0.0,0.0,1.0,1.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
5386,33587874,https://www.airbnb.com/rooms/32244355,0.808449,2,32.74620,-117.16570,Midtown,92103,Cottage,Entire home/apt,3,1.0,1.0,1,Real Bed,120.0,120.0,0.0,15.0,2.0,10.0,1.0,26.0,within a few hours,84.0,0.0,1.0,1.0,1.0,48.0,96.0,96.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,moderate,0.0,0.0


## Simplified Recommendation System

### Load in `sd_trans` dataset

In [16]:
# load in df with the cluster labels
sd_trans = pd.read_csv('data/sd_trans', index_col = 0)
sd_trans.head()

,neighbourhood_cleansed,zipcode,property_type,room_type,bed_type,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,Scripps Ranch,92131,House,Private room,Real Bed,within a few hours,strict_14_with_grace_period,1,1.0,1.0,1,1100.0,1100.0,1000.0,0.0,1.0,0.0,60.0,90.0,100.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,0.0,0.0
1,La Jolla,92037,Apartment,Entire home/apt,Real Bed,no response,strict_14_with_grace_period,1,2.0,2.0,3,1400.0,1400.0,0.0,0.0,1.0,0.0,30.0,1125.0,0.0,0.0,1.0,1.0,0.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,0.0,0.0
2,Pacific Beach,92109,Townhouse,Private room,Real Bed,no response,flexible,1,1.0,1.0,1,1250.0,1250.0,0.0,0.0,1.0,0.0,30.0,30.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,0.0,0.0
3,College Area,92120,Apartment,Entire home/apt,Real Bed,no response,flexible,1,1.0,1.0,1,1150.0,1150.0,0.0,0.0,1.0,0.0,31.0,40.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0
4,La Jolla,92037,Condominium,Private room,Real Bed,within a day,strict_14_with_grace_period,1,1.0,1.0,1,110.0,110.0,200.0,0.0,1.0,50.0,180.0,365.0,67.0,0.0,6.0,1.0,1.0,2.0,4.0,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0


In [17]:
sd_trans.columns

Index(['neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'bed_type', 'host_response_time', 'cancellation_policy', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'nightly_price', 'price_per_stay',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'host_response_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'number_of_reviews',
       'number_of_stays', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'require_guest_profile_picture',
       'require_guest_phone_verification'],
      dtype='object')

In [18]:
features_to_keep = ['neighbourhood_cleansed', 'property_type', 'room_type',
                   'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']

sd_simplified = sd_trans[features_to_keep]
sd_simplified.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,Scripps Ranch,House,Private room,1,1.0,1,1100.0,98.0
1,La Jolla,Apartment,Entire home/apt,1,2.0,3,1400.0,100.0
2,Pacific Beach,Townhouse,Private room,1,1.0,1,1250.0,97.0
3,College Area,Apartment,Entire home/apt,1,1.0,1,1150.0,98.0
4,La Jolla,Condominium,Private room,1,1.0,1,110.0,90.0


### Construct a new preprocessing pipeline

In [19]:
# define nominal and ordinal features in the categorical columns
nom_cols = sd_simplified.select_dtypes(['object']).columns
print(nom_cols)

ordinal_cols = sd_simplified.select_dtypes(['category']).columns
print(ordinal_cols)

Index(['neighbourhood_cleansed', 'property_type', 'room_type'], dtype='object')
Index([], dtype='object')


We only have numeric and nominal features to preprocess in this simplified dataframe. 

In [20]:
# define numeric transformation pipeline that scales the numbers
numeric_pipeline = Pipeline([('numnorm', StandardScaler())]) 


# define a nominal transformation pipeline that OHE the cats
nominal_pipeline = Pipeline([('onehotenc', OneHotEncoder(categories= "auto", 
                                                         sparse = False, 
                                                         handle_unknown = 'ignore'))]) 

In [21]:
# construct column transformer for the selected columns with pipelines
ct = ColumnTransformer(transformers = [("nominalpipe", nominal_pipeline, ['neighbourhood_cleansed', 
                                                                          'property_type', 'room_type']),
                                       ("numericpipe", numeric_pipeline, sd_simplified.select_dtypes(['int', 'float']).columns)])



In [22]:
sd_simplified.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,Scripps Ranch,House,Private room,1,1.0,1,1100.0,98.0
1,La Jolla,Apartment,Entire home/apt,1,2.0,3,1400.0,100.0
2,Pacific Beach,Townhouse,Private room,1,1.0,1,1250.0,97.0
3,College Area,Apartment,Entire home/apt,1,1.0,1,1150.0,98.0
4,La Jolla,Condominium,Private room,1,1.0,1,110.0,90.0


### Transforming the simplified dataset

In [23]:
sd_simplified_pp = pd.DataFrame(ct.fit_transform(sd_simplified))

In [24]:
sd_simplified_pp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152181,-0.561263,-0.728248,2.546108,0.372804
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.152181,0.543637,0.323957,3.417825,0.683043
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152181,-0.561263,-0.728248,2.981967,0.217684
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.152181,-0.561263,-0.728248,2.691394,0.372804
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152181,-0.561263,-0.728248,-0.330558,-0.868154


### Construct a custom listing dataframe
Using user input, provide recommendations for a similar listing. 

In [25]:
# create an empty df with news column for input of values for recommendations
input_df = pd.DataFrame(columns = sd_simplified.columns)

In [26]:
# input numerical value for each predictor, convert to int value
neighborhood_input = input("Enter Neighborhood: ")
property_input = input("Enter Property Type: ")
room_input = input("Enter Room Type: ")
accommodates = int(input("Enter Accommodation Number: "))
bathrooms = int(input("Number of Bathrooms: "))
beds = int(input("Number of Beds: "))
price = int(input("Nightly Price ($): "))
rating = int(input("Review Score Rating: "))

# assign input values to input_df columns
input_df.loc[0, 'neighbourhood_cleansed' ] = neighborhood_input
input_df.loc[0, 'property_type'] = property_input
input_df.loc[0, 'room_type' ] = room_input
input_df.loc[0, 'accommodates'] = accommodates
input_df.loc[0, 'bathrooms' ] = bathrooms
input_df.loc[0, 'beds' ] = beds
input_df.loc[0, 'nightly_price'] = price
input_df.loc[0, 'review_scores_rating' ] = rating

# convert to type float, int
input_df = input_df.astype({"accommodates":"int",
                            "bathrooms":"float",
                            "beds":"int",
                            "nightly_price":"float",
                            "review_scores_rating":"float"})

input_df

Enter Neighborhood: La Jolla
Enter Property Type: House
Enter Room Type: Entire home/apt
Enter Accommodation Number: 3
Number of Bathrooms: 2
Number of Beds: 2
Nightly Price ($): 300
Review Score Rating: 95


,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,La Jolla,House,Entire home/apt,3,2.0,2,300.0,95.0


### Get Recommendations for Simplified Dataset

In [33]:
def get_simplified_recommendations(df, listing):
    """
    Takes in preprocessed dataframe and preprocessed custom listing as inputs and gives top 5
    recommendations based on cosine similarity. 
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # preprocess the listing input
    listing_pp = pd.DataFrame(ct.transform(listing))
    
    # convert single listing to an array
    listing_array = listing_pp.values
    # convert all listings to an array
    df_array = df.values
    
    # get arrays into a single dimension
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # compute cosine similarity 
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # add similarity into recommendations df and reset the index
    rec = sd_simplified.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # add in listings_urls
    # merge on index
    rec = rec.join(sd_listings_url)
    
    # reorder column names
    rec = rec[['listing_url', 'similarity', 'neighbourhood_cleansed', 'property_type', 
               'room_type', 'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(5)


In [34]:
get_simplified_recommendations(sd_simplified_pp, input_df)

,listing_url,similarity,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
6298,https://www.airbnb.com/rooms/25806580,0.966059,La Jolla,House,Entire home/apt,4,2.0,2,175.0,95.0
5765,https://www.airbnb.com/rooms/20898305,0.954252,La Jolla,House,Entire home/apt,4,2.0,2,425.0,97.0
8665,https://www.airbnb.com/rooms/17975477,0.914229,La Jolla,House,Entire home/apt,5,2.0,2,411.0,93.0
6504,https://www.airbnb.com/rooms/9737716,0.912251,La Jolla,House,Entire home/apt,4,2.0,2,169.0,99.0
5933,https://www.airbnb.com/rooms/33222532,0.901171,La Jolla,House,Entire home/apt,4,2.5,3,400.0,97.0
